In [53]:
from dotenv import find_dotenv, load_dotenv
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from typing import Optional
from pydantic import BaseModel, Field, ValidationError
from langchain_core.runnables import RunnableLambda, RunnableMap
load_dotenv(find_dotenv())

True

In [15]:
llm = ChatOpenAI(model='gpt-4o-mini')

In [60]:
class OrderQuery(BaseModel):
    order_id: Optional [int]= None
    product_name: Optional [str] =None
    issue_type: str = None
    details: Optional[str] = None



parser = PydanticOutputParser(pydantic_object= OrderQuery)
format_inst = parser.get_format_instructions()
format_inst

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"order_id": {"anyOf": [{"type": "integer"}, {"type": "null"}], "default": null, "title": "Order Id"}, "product_name": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "title": "Product Name"}, "issue_type": {"default": null, "title": "Issue Type", "type": "string"}, "details": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "title": "Details"}}}\n```'

In [61]:

user_query_prompt =ChatPromptTemplate.from_messages(
                                            [
                                                ("system",
                                            "You are and AI assistant for a e-commerce company, "
                                            "You MUST extract the structured information from the customer's query. \n\n"
                                            "{format_instructions}"
                                                ),
                                          ("human",
                                           "query: {query}"
                                          )]
)
                                           


In [62]:
# chain:
extracted_query_chain = user_query_prompt | llm | parser

In [63]:
test_queries = [
    'Where is my order #23212',
    'Suggest me a good action camera under 10K',
    'I want to return my order #91223, It is not working properly'
]
for i in test_queries:
    output = extracted_query_chain.invoke({'query':i, 'format_instructions': parser.get_format_instructions()})
    print(output)

order_id=23212 product_name=None issue_type='Order Status Inquiry' details=None
order_id=None product_name='action camera' issue_type='suggestion' details='Find a good action camera under 10K.'
order_id=91223 product_name=None issue_type='return' details='It is not working properly'


In [ ]:
def _handle_validation_error(error: Exception) -> dict:
    """Formats a ValidationError into a structured error dictionary."""
    # Extract relevant error details, e.g., error.errors()
    return {
        "success": False,
        "order_query": None,
        "error": f"Validation failed: {error}"
    }

# Convert the error handler function into a RunnableLambda
validation_error_handler = RunnableLambda(_handle_validation_error)

def _format_final_output(result):
    """Formats the final output to be consistent for success or error cases."""
    if isinstance(result, OrderQuery):
        return {"success": True, "order_query": result, "error": None}
    else:  # This branch is for the error dict from validation_error_handler
        return result

# Replace the previous validation_step with a final formatter
final_output_formatter = RunnableLambda(_format_final_output)

# The order_validation_chain now uses with_fallbacks to catch ValidationErrors
# and routes successful outputs or handled errors to the final formatter.
order_validation_chain = (
    user_query_prompt
    | llm
    | parser.with_fallbacks(
        fallbacks=[validation_error_handler],
        exceptions_to_handle=(ValidationError,)
    )
    | final_output_formatter # This step ensures consistent output format
)
     
# Test with a working query (LLM should correctly provide issue_type)
order_validation_chain.invoke({'query':'Where is my order #23212', 'format_instructions': parser.get_format_instructions()})

{'success': True,
 'order_query': OrderQuery(order_id=23212, product_name=None, issue_type='Shipping Inquiry', details=None),
 'error': None}

: 